## Reddit threads API
Install PRAW `(pip install praw)`. In your backend code, initialize PRAW with your credentials:
- PRAW requires client_id, client_secret, and user_agent. 

In [19]:
# Load Env variables
from dotenv import load_dotenv
load_dotenv()

True

In [1]:
import os
import praw

REDDIT_CLIENTID = os.getenv('REDDIT_CLIENTID')
REDDIT_SECRET = os.getenv('REDDIT_SECRET')
REDDIT_USER_AGENT = os.getenv('REDDIT_USER_AGENT')


reddit = praw.Reddit(
    client_id=REDDIT_CLIENTID,
    client_secret=REDDIT_SECRET,
    user_agent=REDDIT_USER_AGENT,
)

print(reddit.read_only)

True


​Once initialized (in read-only mode), you can fetch submissions. For example, to get the latest posts from r/wallstreetbets:

In [2]:
subreddit = reddit.subreddit("wallstreetbets")
posts = subreddit.search("TSLA", time_filter="week", limit=50)

Or simply `reddit.subreddit("stocks").hot(limit=20)`​. This yields Submission objects from which you can extract titles, selftext, and comments.

Within your API endpoint (or a helper function), query relevant subreddits for the given stock ticker or company name. You might search multiple subreddits (e.g. r/stocks, r/investing) and combine results. For each submission, gather its title and top comments (e.g. use submission.comments). Filter out removed or very short comments. Collect these texts into a list for analysis.

## Polygon IO API
Install polygon-api-client `pip install -U polygon-api-client`. Use it to get historical price data for the ticker. For example:

In [17]:
from polygon import RESTClient

POLYGON_API_KEY = os.getenv("POLYGON_API_KEY")

client = RESTClient(POLYGON_API_KEY)

aggs = client.get_aggs(
    "AAPL",
    1,
    "day",
    "2024-04-04",
    "2024-04-10",
)
print(aggs)


[Agg(open=170.29, high=171.92, low=168.82, close=168.82, volume=53682486.0, vwap=170.1029, timestamp=1712203200000, transactions=630783, otc=None), Agg(open=169.59, high=170.39, low=168.95, close=169.58, volume=42104826.0, vwap=169.6415, timestamp=1712289600000, transactions=540854, otc=None), Agg(open=169.03, high=169.2, low=168.24, close=168.45, volume=37425513.0, vwap=168.6637, timestamp=1712548800000, transactions=549987, otc=None), Agg(open=168.7, high=170.08, low=168.35, close=169.67, volume=42451209.0, vwap=169.1566, timestamp=1712635200000, transactions=541699, otc=None), Agg(open=168.8, high=169.09, low=167.11, close=167.78, volume=49691936.0, vwap=167.9914, timestamp=1712721600000, transactions=647587, otc=None)]


## OpenAI
We want an NLG summary, not just labels. A good approach is to use a transformer summarization pipeline (abstractive summarization). Install OpenAI `(pip install openai)`. For example:

In [21]:
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.responses.create(
    model="gpt-4o",
    instructions="Please provide a natural language summary of the current sentiment towards TSLA based on the discussions.",
    input="TSLA has been down a lot. TSLA is great right now. Elon Musk is selling TSLA",
)

print(response.output_text)

The sentiment towards TSLA appears mixed. While some people recognize that the stock has been down significantly, others feel positively about its current state. There's also concern or interest about Elon Musk selling TSLA shares, which may be impacting perspectives.
